<a href="https://colab.research.google.com/github/luasampaio/data-engineering/blob/main/54_explode_vs_explode_outer_em_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## explode vs explode_outer em PySpark

-  explode() para remover linhas sem elementos (arrays vazios/nulos).
-  explode_outer() quando quer manter todas as linhas, mesmo que não haja elementos para explodir.
- posexplode_outer() cria linhas para cada elemento do array, trazendo também a posição.

---
- Data: 22/07/25
- Luciana Sampaio Mayer



In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, ArrayType
from pyspark.sql.functions import explode, explode_outer, posexplode, posexplode_outer

# 1. Cria sessão Spark
spark = SparkSession.builder \
    .appName("explode") \
    .master("local") \
    .getOrCreate()

In [2]:
# 2. Dados de entrada
df = [
    ("1", ["Matematica", "fisica", "Quimica"]),
    ("2", ["Ingles", "Frances", "Alemao"]),
    ("3", ["Portugues"]),
    ("4", []),
    ("5", None)
]

Configurando o schema

In [3]:
# 3. Cria um esquema para os dados
schema = StructType([
    StructField("id", StringType(), True),
    StructField("materias", ArrayType(StringType()), True)
])

# Converte a lista Python para um Spark DataFrame
spark_df = spark.createDataFrame(df, schema=schema)



In [4]:
# 4. Mostra o esquema do DataFrame
spark_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- materias: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [5]:
# 5.visualiza os dados
spark_df.show()

+---+--------------------+
| id|            materias|
+---+--------------------+
|  1|[Matematica, fisi...|
|  2|[Ingles, Frances,...|
|  3|         [Portugues]|
|  4|                  []|
|  5|                NULL|
+---+--------------------+



In [6]:
# 6. Explode a coluna 'Materias'
explode = spark_df.withColumn("Disciplina", explode("Materias"))

In [7]:
print("Usando explode():")
explode.show(truncate=False)

Usando explode():
+---+-----------------------------+----------+
|id |materias                     |Disciplina|
+---+-----------------------------+----------+
|1  |[Matematica, fisica, Quimica]|Matematica|
|1  |[Matematica, fisica, Quimica]|fisica    |
|1  |[Matematica, fisica, Quimica]|Quimica   |
|2  |[Ingles, Frances, Alemao]    |Ingles    |
|2  |[Ingles, Frances, Alemao]    |Frances   |
|2  |[Ingles, Frances, Alemao]    |Alemao    |
|3  |[Portugues]                  |Portugues |
+---+-----------------------------+----------+



# explode_outer()

In [8]:
explode_outer = spark_df.withColumn("Disciplina", explode_outer("Materias"))
explode_outer.show(truncate=False)

+---+-----------------------------+----------+
|id |materias                     |Disciplina|
+---+-----------------------------+----------+
|1  |[Matematica, fisica, Quimica]|Matematica|
|1  |[Matematica, fisica, Quimica]|fisica    |
|1  |[Matematica, fisica, Quimica]|Quimica   |
|2  |[Ingles, Frances, Alemao]    |Ingles    |
|2  |[Ingles, Frances, Alemao]    |Frances   |
|2  |[Ingles, Frances, Alemao]    |Alemao    |
|3  |[Portugues]                  |Portugues |
|4  |[]                           |NULL      |
|5  |NULL                         |NULL      |
+---+-----------------------------+----------+



In [9]:
spark_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- materias: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [10]:
print("Usando posexplode_outer():")
posexplode_outer_df = spark_df.select("id", posexplode_outer("materias").alias("pos", "Disciplina"))
posexplode_outer_df.show(truncate=False)

Usando posexplode_outer():
+---+----+----------+
|id |pos |Disciplina|
+---+----+----------+
|1  |0   |Matematica|
|1  |1   |fisica    |
|1  |2   |Quimica   |
|2  |0   |Ingles    |
|2  |1   |Frances   |
|2  |2   |Alemao    |
|3  |0   |Portugues |
|4  |NULL|NULL      |
|5  |NULL|NULL      |
+---+----+----------+



Explicação:

- posexplode_outer("Materias"): para cada elemento do array da coluna Materias,cria uma nova linha com duas informações:

   - O índice (posição) do elemento (pos)

   - O valor do elemento (col)

> Para linhas onde Materias é vazio ou nulo, retorna uma linha com null nessas colunas.

O resultado é uma nova coluna Disciplina, que é uma struct com pos e col.

In [11]:
posexplode_outer_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- pos: integer (nullable = true)
 |-- Disciplina: string (nullable = true)

